In [ ]:
import os
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras import utils
from keras.preprocessing.text import Tokenizer
import numpy as np

imdb_dir = '/content/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts  =[]

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

# Токенизация текста из исходного набора данных
maxlen = 100 # Отсечение остатка отзывовпосле 100 слова
training_samples = 200 # Обучение на выборке из 200 образцов
validation_samples = 10000 # Проверка на выборке из 10 000 образцов
max_words = 10000 # Рассм только 10 тыс наиболее часто использ-х слов

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = utils.data_utils.pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor',data.shape)
print('Shape of label tensor', labels.shape)

#Разбивка данных на обучающую и проверочные выборки, но перед этим данные перемешиваются
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples+validation_samples]
y_val = labels[training_samples: training_samples+validation_samples]


# Теперь векторное представление Glove word-embedding file 
glove_dir = ''
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove-6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors' %len(embeddings_index))

# Подготовка матрицы векторных представлений слов glove
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i<max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None :
            # Словам, кот нет в индексе представлений, будут соотв-ть нулвые векторы
            embedding_matrix[i]  = embedding_vector
    
# Определение модели
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# Загрузка предварительно обученных векторных представлений слов в слой еmbeddings
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False # заморозка слоя

# Обучение и оценка модели
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val)
)
model.save_weights('pre_trained_glove_model.h5')

# Вывод результатов
import matplotlib.pyplot as plt 
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Обучение той же модели без обученных векторных представлений 
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense 

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)
history= model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val)
)

# Токенизация данных из контрольной выборки 
test_dir = os.path.join(imdb_dir, 'test')
labels = []
texts = []

for label_type in ['neg','pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] =='.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type=='neg':
                labels.append[0]
            else:
                labels.append[1]

sequences = tokenizer.texts_to_sequences[texts]
x_test = utils.data_utils.pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

# Оценка модели на контрольном наборе данных
model.load_weights('pre-trained_glove_model.h5')
model.evaluate(x_test, y_test)